# Machine Learning and Predictive Analytics: Solving a Real World Problem with Machine Learning

## Datasets: Los Angeles Crime Data 2010-19 and 2020-25
2010-19 dataset: https://catalog.data.gov/dataset/crime-data-from-2010-to-2019      
2020-25 dataset (accessed up to 25/05/2025): https://catalog.data.gov/dataset/crime-data-from-2020-to-present

## Research Question: How can we protect children from being victims of crime in Los Angeles?

The model will predict the risk level of a child becoming a victim of crime, based on demographic factors (such as age, sex, and descent) in combination with spatial and temporal variables (such as location, time of day, and day of the week).

Real-world interventions can be based on the predictions of the model. For example, if the model predicts that there is a high risk level for Black children being victimised in Central LA during weekday evenings, a local youth centre could implement targeted outreach programmes during those hours — offering safe spaces, support services, or structured activities.

## Methodology Plan

### Preparation
- Combine and clean two datasets
    - ~~Rename columns for clarity~~
    - ~~Rename/group low frequency values in `vict_sex` and `vict_descent`~~
    - ~~Convert messy dates and times~~
    - Categorise children as those under 18 and over 0   

### Questions to explore
- Where are children most likely to be victims of crime?
- When are children most likely to be victims of crime? - time of year, day of week, hour of day
    - Issue with logging of dates - crimes disproportionately logged on 1st Jan or first of month - can use day of week as a proxy
    - Look at metadata to understand times - is there enough accuracy to use this variable?
- How are demographics associated with crime and is this a useful factor to include?

### Visualisations
- Make heatmap 

### Building the model
- Decide on which features to include
- One-hot encode categorical variables
- Split data into training and test sets
- Group crime types to reduce dimensionality

**Problems with temporal variables:**
- Dates are likely to contain inaccuracies - crimes being logged on 1st Jan or first of month
- Times are stored as text but military times can start with 0s so there are errors
    - Cleaning is possible to a certain extent but would require assumptions (e.g. is '4' supposed to be 4am, 4pm, or a typo?)
- Times may be inaccurate if officers wait until their shift ends to log crimes (may explain peaks around midday, 6pm etc)
- Temporal data is not stored as datetime


## Setup and Pre-processing

In [1]:
#import libraries
import pandas as pd
import numpy as np
import janitor 

In [2]:
#get 2010-19 data from csv
df1 = pd.read_csv("la_crimes_2010-19.csv")

#get 2020-25 data from csv
df2 = pd.read_csv("la_crimes_2020-25.csv")

#clean variable names
df1 = (
    df1.clean_names()
    .rename(columns={"date_occ":"date", "time_occ":"time", "area_name":"area", "crm_cd":"crime_code", "crm_cd_desc":"crime_type", "premis_cd":"premises_code", "premis_desc":"premises_type", "weapon_used_cd":"weapon_code", "weapon_desc":"weapon_type"})
    )

df2 = (
    df2.clean_names()
    .rename(columns={"date_occ":"date", "time_occ":"time", "area":"area_", "area_name":"area", "crm_cd":"crime_code", "crm_cd_desc":"crime_type", "premis_cd":"premises_code", "premis_desc":"premises_type", "weapon_used_cd":"weapon_code", "weapon_desc":"weapon_type"})
    )

#join dataframes and view all columns
df = pd.concat([df1, df2], ignore_index=True)
pd.set_option('display.max_columns', None)
df.head(4)

,dr_no,date_rptd,date,time,area_,area,rpt_dist_no,part_1_2,crime_code,crime_type,mocodes,vict_age,vict_sex,vict_descent,premises_code,premises_type,weapon_code,weapon_type,status,status_desc,crm_cd_1,crm_cd_2,crm_cd_3,crm_cd_4,location,cross_street,lat,lon
0,1307355,02/20/2010 12:00:00 AM,02/20/2010 12:00:00 AM,1350,13,Newton,1385,2,900,VIOLATION OF COURT ORDER,0913 1814 2000,48,M,H,501.0,SINGLE FAMILY DWELLING,NaN,NaN,AA,Adult Arrest,900.0,NaN,NaN,NaN,300 E GAGE AV,NaN,33.9825,-118.2695
1,11401303,09/13/2010 12:00:00 AM,09/12/2010 12:00:00 AM,45,14,Pacific,1485,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",0329,0,M,W,101.0,STREET,NaN,NaN,IC,Invest Cont,740.0,NaN,NaN,NaN,SEPULVEDA BL,MANCHESTER AV,33.9599,-118.3962
2,70309629,08/09/2010 12:00:00 AM,08/09/2010 12:00:00 AM,1515,13,Newton,1324,2,946,OTHER MISCELLANEOUS CRIME,0344,0,M,H,103.0,ALLEY,NaN,NaN,IC,Invest Cont,946.0,NaN,NaN,NaN,1300 E 21ST ST,NaN,34.0224,-118.2524
3,90631215,01/05/2010 12:00:00 AM,01/05/2010 12:00:00 AM,150,6,Hollywood,646,2,900,VIOLATION OF COURT ORDER,1100 0400 1402,47,F,W,101.0,STREET,102.0,HAND GUN,IC,Invest Cont,900.0,998.0,NaN,NaN,CAHUENGA BL,HOLLYWOOD BL,34.1016,-118.3295


### Data Cleaning

**NOTES:**  
- It is fair to assume that `vict_sex` and `vict_descent` will be important features 
- Viewing the unique values in `vict_descent` shows that there are several low-frequency values. Knowing that this may be included in later analyses, and to avoid anonymisation issues, I will combine values with fewer than 30 observations into "Other". This approach will help protect the validity of future tests and models; multiple low-frequency groups in this context would represent too small a sample size for each value to draw meaningful conclusions (Krishnan, 2011).
- Times are stored as ints but military times can start with 0s so there are errors. Cleaning is possible to a certain extent but would require assumptions (e.g. is '4' supposed to be 4am, 4pm, or a typo?). For this reason, I will remove times that are less than 100. For three-digit times, I will add a leading 0 to return them to military time. I will then convert to a datetime object.
- I will drop rows with missing victim age or where age is zero, as this variable is essential for building my model. (vict_age contains many 0s, possibly as crimes without known/human victims e.g. vandalism) 

In [3]:
#tidy victim sex variable
df["vict_sex"] = df["vict_sex"].replace(["M", "F", "X", "H", "N", "-"], ["Male", "Female", "Other/Unknown", "Other/Unknown", "Other/Unknown", "Other/Unknown"])

#tidy victim descent variable
descent_map = {
    "A": "Other Asian",
    "B": "Black",
    "C": "Chinese",
    "D": "Cambodian",
    "F": "Filipino",
    "G": "Guamanian",
    "H": "Hispanic/Latin/Mexican",
    "I": "American Indian/Alaskan Native",
    "J": "Japanese",
    "K": "Korean",
    "L": "Laotian",
    "O": "Other",
    "P": "Pacific Islander",
    "S": "Samoan",
    "U": "Hawaiian",
    "V": "Vietnamese",
    "W": "White",
    "X": "Unknown",
    "Z": "Asian Indian",
    "-": "Unknown"
}
df["vict_descent"] = df["vict_descent"].replace(descent_map)

In [ ]:
#convert dates
df["date_rptd"] = pd.to_datetime(df["date_rptd"], format="%m/%d/%Y %I:%M:%S %p").dt.normalize()
df["date"] = pd.to_datetime(df["date"], format="%m/%d/%Y %I:%M:%S %p").dt.normalize()

#convert times
df = df[df["time"] > 99]
df["time"] = df["time"].astype(str).str.zfill(4)

#get datetime column
df["datetime_str"] = df["date"].dt.strftime("%Y-%m-%d") + " " + df["time"].str[:2] + ":" + df["time"].str[2:]
df["datetime"] = pd.to_datetime(df["datetime_str"], format="%Y-%m-%d %H:%M")
df.drop(columns="datetime_str", inplace=True)

In [ ]:
#drop columns that won't be used for the model
df = df.drop(columns=["dr_no", "date_rptd", "area_", "rpt_dist_no", "part_1_2", "crime_code", "mocodes", "premises_code", "weapon_code", "status", "status_desc", "crm_cd_1", "crm_cd_2", "crm_cd_3", "crm_cd_4", "location", "cross_street"])

#drop rows with missing victim age
df = df.dropna(subset=["vict_age"])

#drop rows where age is zero
df = df[df["vict_age"] != 0]

In [ ]:
df.head(4)

### Grouping categorical data

#### Weapon Type

There are 80 unique weapon types in the dataset. I felt that this was a small enough number to handle mostly manually, so I defined categories that I felt to be logical. For efficienct, I asked ChatGPT to categorise each of the weapon types into my pre-defined categories (OpenAI, 2025). I edited the dictionary slightly to tweak the decisions made by ChatGPT, to ensure that the categorisation was logical, e.g. changing "SYRINGE" from "Burning/Toxic Substance" to "Knife/Blade/Sharp Object".

In [ ]:
#define weapon categories
weapon_map = {
    "STRONG-ARM (HANDS, FIST, FEET OR BODILY FORCE)": "Bodily Force",
    "UNKNOWN WEAPON/OTHER WEAPON": "Other/Unknown/No Weapon Used",
    "VERBAL THREAT": "Verbal Threat",
    "HAND GUN": "Gun/Firearm",
    "KNIFE WITH BLADE 6INCHES OR LESS": "Knife/Blade/Sharp Object",
    "SEMI-AUTOMATIC PISTOL": "Gun/Firearm",
    "OTHER KNIFE": "Knife/Blade/Sharp Object",
    "UNKNOWN FIREARM": "Gun/Firearm",
    "VEHICLE": "Vehicle",
    "MACE/PEPPER SPRAY": "Burning/Toxic Substance",
    "BOTTLE": "Blunt/Hitting Object",
    "STICK": "Blunt/Hitting Object",
    "ROCK/THROWN OBJECT": "Blunt/Hitting Object",
    "CLUB/BAT": "Blunt/Hitting Object",
    "FOLDING KNIFE": "Knife/Blade/Sharp Object",
    "REVOLVER": "Gun/Firearm",
    "KITCHEN KNIFE": "Knife/Blade/Sharp Object",
    "BLUNT INSTRUMENT": "Blunt/Hitting Object",
    "KNIFE WITH BLADE OVER 6 INCHES IN LENGTH": "Knife/Blade/Sharp Object",
    "PIPE/METAL PIPE": "Blunt/Hitting Object",
    "AIR PISTOL/REVOLVER/RIFLE/BB GUN": "Gun/Firearm",
    "SIMULATED GUN": "Gun/Firearm",
    "BELT FLAILING INSTRUMENT/CHAIN": "Blunt/Hitting Object",
    "OTHER CUTTING INSTRUMENT": "Knife/Blade/Sharp Object",
    "HAMMER": "Blunt/Hitting Object",
    "PHYSICAL PRESENCE": "Bodily Force",
    "SCREWDRIVER": "Knife/Blade/Sharp Object",
    "MACHETE": "Knife/Blade/Sharp Object",
    "UNKNOWN TYPE CUTTING INSTRUMENT": "Knife/Blade/Sharp Object",
    "SCISSORS": "Knife/Blade/Sharp Object",
    "OTHER FIREARM": "Gun/Firearm",
    "CONCRETE BLOCK/BRICK": "Blunt/Hitting Object",
    "SHOTGUN": "Gun/Firearm",
    "RIFLE": "Gun/Firearm",
    "FIXED OBJECT": "Blunt/Hitting Object",
    "STUN GUN": "Gun/Firearm",
    "BOARD": "Blunt/Hitting Object",
    "FIRE": "Burning/Toxic Substance",
    "GLASS": "Blunt/Hitting Object",
    "SWITCH BLADE": "Knife/Blade/Sharp Object",
    "CAUSTIC CHEMICAL/POISON": "Burning/Toxic Substance",
    "BRASS KNUCKLES": "Blunt/Hitting Object",
    "AXE": "Knife/Blade/Sharp Object",
    "TIRE IRON": "Blunt/Hitting Object",
    "SCALDING LIQUID": "Burning/Toxic Substance",
    "TOY GUN": "Gun/Firearm",
    "RAZOR BLADE": "Knife/Blade/Sharp Object",
    "SWORD": "Knife/Blade/Sharp Object",
    "BOMB THREAT": "Verbal Threat",
    "RAZOR": "Knife/Blade/Sharp Object",
    "ICE PICK": "Knife/Blade/Sharp Object",
    "HECKLER & KOCH 93 SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm",
    "ASSAULT WEAPON/UZI/AK47/ETC": "Gun/Firearm",
    "DIRK/DAGGER": "Knife/Blade/Sharp Object",
    "LIQUOR/DRUGS": "Other/Unknown/No Weapon Used",
    "EXPLOXIVE DEVICE": "Burning/Toxic Substance",
    "AUTOMATIC WEAPON/SUB-MACHINE GUN": "Gun/Firearm",
    "SAWED OFF RIFLE/SHOTGUN": "Gun/Firearm",
    "STARTER PISTOL/REVOLVER": "Gun/Firearm",
    "ROPE/LIGATURE": "Other/Unknown/No Weapon Used",
    "SEMI-AUTOMATIC RIFLE": "Gun/Firearm",
    "CLEAVER": "Knife/Blade/Sharp Object",
    "BOWIE KNIFE": "Knife/Blade/Sharp Object",
    "DOG/ANIMAL (SIC ANIMAL ON)": "Other/Unknown/No Weapon Used",
    "DEMAND NOTE": "Verbal Threat",
    "STRAIGHT RAZOR": "Knife/Blade/Sharp Object",
    "BLACKJACK": "Blunt/Hitting Object",
    "SYRINGE": "Knife/Blade/Sharp Object",
    "BOW AND ARROW": "Other/Unknown/No Weapon Used",
    "MARTIAL ARTS WEAPONS": "Blunt/Hitting Object",
    "UNK TYPE SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm",
    "UZI SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm",
    "RELIC FIREARM": "Gun/Firearm",
    "HECKLER & KOCH 91 SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm",
    "ANTIQUE FIREARM": "Gun/Firearm",
    "MAC-10 SEMIAUTOMATIC ASSAULT WEAPON": "Gun/Firearm",
    "MAC-11 SEMIAUTOMATIC ASSAULT WEAPON": "Gun/Firearm",
    "M1-1 SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm",
    "M-14 SEMIAUTOMATIC ASSAULT RIFLE": "Gun/Firearm"
}

#map to dataframe, drop original column
df["weapon_group"] = df["weapon_type"].map(weapon_map).fillna("Other/Unknown/No Weapon Used")
df.drop("weapon_type", axis=1, inplace=True)


#### Crime Type

There are 142 crime types in the dataset, which I felt was too many to deal with completely manually, but I noticed that many of them had repeating words (e.g. "THEFT") so I wrote a function to group them by keyword. 

In [ ]:
#define function to group crimes
def crime_grouping(crime):
    crime = crime.upper()
    if any(word in crime for word in ["CHILD"]):
        return "Offense Against a Child"
    elif any(word in crime for word in ["HOMICIDE", "MANSLAUGHTER", "LYNCHING"]):
        return "Murder/Manslaughter"
    elif any(word in crime for word in ["ASSAULT", "BRANDISH", "SHOTS", "BATTERY", "BOMB"]):
        return "Assault/Violence"
    elif any(word in crime for word in ["THEFT", "BURGLARY", "ROBBERY", "STOLEN", "EXTORTION", "PICKPOCKET", "SNATCHING", "BUNCO", "FRAUD", "COUNTERFEIT"]):
        return "Theft-Related"
    elif any(word in crime for word in ["VANDALISM", "ARSON"]):
        return "Property Damage"
    elif any(word in crime for word in ["VIOLATION", "TRESPASSING", "DISTURBING", "CONTEMPT", "THROWING", "RESISTING", "STALKING", "PROWLER", "THREAT"]):
        return "Public Order/Threatening Behaviour"
    elif any(word in crime for word in ["LEWD", "SEX", "RAPE", "PENETRATION", "INDECENT", "COPULATION", "PEEPING", "PIMPING"]):
        return "Sexual Offence"
    elif any(word in crime for word in ["KIDNAPPING", "IMPRISONMENT", "TRAFFICKING"]):
        return "Kidnapping/Trafficking"
    else:
        return "Other"

#call function, apply to dataframe, drop original column
df["crime_group"] = df["crime_type"].apply(crime_grouping)
df.drop("crime_type", axis=1, inplace=True)